# Set initialization file
https://cccma.gitlab.io/classic/makeInputFiles.html

In [1]:
# Env: sc2_v0

import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt # New plot library (https://proplot.readthedocs.io/en/latest/)
pplt.rc['savefig.dpi'] = 300 # 1200 is too big! #https://proplot.readthedocs.io/en/latest/basics.html#Creating-figures

In [2]:
site = 'oas'
site_ex = 'CA-Oas'
# site_ex = 'AU-Tum'

path_in = '/home/lalandmi/eccc/classic-develop/inputFiles/FLUXNETsites/'+site_ex # example file
path_out = '/home/lalandmi/eccc/classic-develop/inputFiles/SnowMIP/'+site

In [3]:
ds = xr.open_dataset(path_in+'/'+site_ex+'_init.nc')
ds.load()

<xarray.Dataset>
Dimensions:            (tile: 1, lat: 1, lon: 1, icp1: 6, layer: 20, ic: 5,
                        icc: 12, iccp1: 13, iccp2: 14, slope: 8)
Coordinates:
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * icp1               (icp1) int32 1 2 3 4 5 6
  * lat                (lat) float64 53.63
  * layer              (layer) int32 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
  * lon                (lon) float64 -106.2
  * slope              (slope) int32 1 2 3 4 5 6 7 8
  * tile               (tile) int32 1
Data variables: (12/101)
    ALBS               (tile, lat, lon) float64 0.7647
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.29 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.05 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 15.16 20.08 ... 23.29
    CMAS               (tile, ic, lat, lon) float64 0.0 20.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    ...                 ...
    stemmass_s         (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    stmhrlos           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    tcoldm             (lat, lon) float32 99.0
    twarmm             (lat, lon) float32 -99.0
    tymaxlai           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    title:          CLASSIC initialization file created from: /home/acrp001/m...
    creation_date:  20190523
    history:        Wed Jun 12 17:19:01 2019: ncks -4 ./rsfile.nc ./rsfile.nc
    NCO:            4.4.2
    row_bounds:     1 1

In [4]:
ds.soilcmas.sum()

<xarray.DataArray 'soilcmas' ()>
array(0.)

## BERMS Old Aspen, Saskatchewan, Canada

Even-aged stand of aspen with a thick hazelnut understorey, naturally regenerated after
fire in 1919. 10 cm layer of organic litter and peat over a sandy clay loam.

| Short name | oas |
|:-----------|:----|
| Location | 53.63ºN, 106.20ºW |
| Elevation | 600 m |
| Canopy height | 21 m |
| Leaf area index | winter stem area ~1, summer 3.7 – 5.2 |
| Snow-free albedo | 0.14 |
| Simulation period | 1 October 1997 to 30 September 2010 |
| Temperature/humidity measurement height | 37 m |
| Wind measurement height | 38 m |
| Reference | Bartlett et al. (2007) |

In [5]:
# If the site description is not enough we can get the input data from gridded satellite datasets
path = '/home/lalandmi/Dropbox/data/CLASSIC/'
SoilGrids250m_CLAY = xr.open_dataset(path+'/soil/SoilGrids250m_CLAY_0.05deg.nc')
SoilGrids250m_ORGM = xr.open_dataset(path+'/soil/SoilGrids250m_ORGM_0.05deg.nc')
SoilGrids250m_SAND = xr.open_dataset(path+'/soil/SoilGrids250m_SAND_0.05deg.nc')
SoilGrids250m_SDEP = xr.open_dataset(path+'/soil/SoilGrids250m_SDEP_0.05deg.nc')
NCAR_SOCI = xr.open_dataset(path+'/soil/NCAR_SOCI_0.5deg.nc')
c4_fraction_1deg = xr.open_dataarray(path+'/vegetation/c4_fraction_1deg.nc')

In [6]:
lat = 53.62889
lon = -106.19779

# These values can be attributed with the global grid if not available
SDEP = SoilGrids250m_SDEP.sel(lat=lat, lon=lon, method='nearest').SDEP.values.item(0) # Soil permeable depth (m)
SOCI = NCAR_SOCI.sel(lat=lat, lon=lon, method='nearest').SOCI.values.item(0) # Soil color inde
c4_fraction = c4_fraction_1deg.sel(lat=lat, lon=lon, method='nearest').values.item(0) # If grass


# Those can be distributed along the soil layers if available
CLAY = [0.] + [24.2]*2 + [22.1]*(20-3) # Percentage clay content (%)
SAND = [-2.] + [48.8]*2 + [53.6]*(20-3) # Percentage of sand content (%)
ORGM = [77.5] + [0.8]*2 + [4.1]*(20-3) # Percentage organic matter content (%) matter content (%)
# If the sum does not reach 100 %, the left over will be attributed to silt

# CLAY = SoilGrids250m_CLAY.sel(lat=lat, lon=lon, method='nearest').CLAY.values 
# SAND = SoilGrids250m_SAND.sel(lat=lat, lon=lon, method='nearest').SAND.values
# ORGM = SoilGrids250m_ORGM.sel(lat=lat, lon=lon, method='nearest').ORGM.values


# ! List of CLASS-level PFTs
classpfts = ['NdlTr', 'BdlTr', 'Crops', 'Grass', 'BdlSh']
FCAN = {'NdlTr': 0. , 'BdlTr': 1., 'Crops': 0., 'Grass': 0., 'BdlSh': 0.} # max 1
# Maybe add a little fraction of trees (as it's bordered by coniferous -> TODO)

# ! List of CTEM PFTs
# ! **Note: 'BdlDCoTr' should be specified before 'BdlDDrTr' due to some code in competition.
ctempfts = ['NdlEvgTr', 'NdlDcdTr', 'BdlEvgTr', 'BdlDCoTr', 'BdlDDrTr', 'CropC3', 'CropC4', 'GrassC3', 'GrassC4', 'Sedge', 'BdlEvgSh', 'BdlDCoSh']
fcancmx = {'NdlEvgTr': 0., 'NdlDcdTr': 0., 'BdlEvgTr': 0., 'BdlDCoTr': 1., 'BdlDDrTr': 0., 'CropC3': 0., 'CropC4': 0., 
           'GrassC3': 0., 'GrassC4': 0., 'Sedge': 0., 'BdlEvgSh': 0., 'BdlDCoSh': 0.} # Max 1
# Check differences between C3 and C4 at Cdp? (TODO)

if sum(FCAN.values()) > 1: raise Exception("The sum of FCAN values needs to be lower than 1.")
if sum(fcancmx.values()) > 1: raise Exception("The sum of fcanmax values needs to be lower than 1.")

print('SDEP = ' + str(SDEP))
print('\nSOCI = ' + str(SOCI))
print('\nCLAY = ' + str(CLAY))
print('\nSAND = ' + str(SAND))
print('\nORGM = ' + str(ORGM))
print('\nGrass C4 fraction = ' + str(c4_fraction))

SDEP = 35.869998931884766

SOCI = 19.0

CLAY = [0.0, 24.2, 24.2, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1]

SAND = [-2.0, 48.8, 48.8, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6]

ORGM = [77.5, 0.8, 0.8, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1]

Grass C4 fraction = 0.00570000009611249


### Set lat/lon

In [7]:
with xr.set_options(keep_attrs=True):
    ds = ds.assign_coords(lat=(ds.lat*0+lat))
    ds = ds.assign_coords(lon=(ds.lon*0+lon))
ds

<xarray.Dataset>
Dimensions:            (tile: 1, lat: 1, lon: 1, icp1: 6, layer: 20, ic: 5,
                        icc: 12, iccp1: 13, iccp2: 14, slope: 8)
Coordinates:
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * icp1               (icp1) int32 1 2 3 4 5 6
  * lat                (lat) float64 53.63
  * layer              (layer) int32 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
  * lon                (lon) float64 -106.2
  * slope              (slope) int32 1 2 3 4 5 6 7 8
  * tile               (tile) int32 1
Data variables: (12/101)
    ALBS               (tile, lat, lon) float64 0.7647
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.29 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.05 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 15.16 20.08 ... 23.29
    CMAS               (tile, ic, lat, lon) float64 0.0 20.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    ...                 ...
    stemmass_s         (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    stmhrlos           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    tcoldm             (lat, lon) float32 99.0
    twarmm             (lat, lon) float32 -99.0
    tymaxlai           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    title:          CLASSIC initialization file created from: /home/acrp001/m...
    creation_date:  20190523
    history:        Wed Jun 12 17:19:01 2019: ncks -4 ./rsfile.nc ./rsfile.nc
    NCO:            4.4.2
    row_bounds:     1 1

### Set PFTs

In [8]:
for i, pft in enumerate(classpfts):
    ds.FCAN[0, i, 0, 0] = FCAN[pft]
    print(pft + ' -> ' + str(FCAN[pft]*100) + ' %')
    
ds.FCAN[0, :, 0, 0]
# 6th PFT: Bareground (need to be specified if CLASS used without CTEM)

NdlTr -> 0.0 %
BdlTr -> 100.0 %
Crops -> 0.0 %
Grass -> 0.0 %
BdlSh -> 0.0 %


<xarray.DataArray 'FCAN' (icp1: 6)>
array([0., 1., 0., 0., 0., 0.])
Coordinates:
  * icp1     (icp1) int32 1 2 3 4 5 6
    lat      float64 53.63
    lon      float64 -106.2
    tile     int32 1
Attributes:
    units:      -
    long_name:  Annual maximum fractional coverage of modelled area (read in ...

In [9]:
for i, pft in enumerate(ctempfts):
    ds.fcancmx[0, i, 0, 0] = fcancmx[pft]
    print(pft + ' -> ' + str(fcancmx[pft]*100) + ' %')
    
ds.fcancmx[0, :, 0, 0]

NdlEvgTr -> 0.0 %
NdlDcdTr -> 0.0 %
BdlEvgTr -> 0.0 %
BdlDCoTr -> 100.0 %
BdlDDrTr -> 0.0 %
CropC3 -> 0.0 %
CropC4 -> 0.0 %
GrassC3 -> 0.0 %
GrassC4 -> 0.0 %
Sedge -> 0.0 %
BdlEvgSh -> 0.0 %
BdlDCoSh -> 0.0 %


<xarray.DataArray 'fcancmx' (icc: 12)>
array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
Coordinates:
  * icc      (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
    lat      float64 53.63
    lon      float64 -106.2
    tile     int32 1
Attributes:
    units:      -
    long_name:  PFT fractional coverage per grid cell

### Set soil color

In [10]:
with xr.set_options(keep_attrs=True):
    ds['SOCI'][0, 0, 0] = SOCI
ds.SOCI

<xarray.DataArray 'SOCI' (tile: 1, lat: 1, lon: 1)>
array([[[19.]]])
Coordinates:
  * lat      (lat) float64 53.63
  * lon      (lon) float64 -106.2
  * tile     (tile) int32 1
Attributes:
    units:      index
    long_name:  Soil colour index

### Set permeable depth

In [11]:
with xr.set_options(keep_attrs=True):
    ds['SDEP'][0, 0, 0] = SDEP
ds.SDEP

<xarray.DataArray 'SDEP' (tile: 1, lat: 1, lon: 1)>
array([[[35.86999893]]])
Coordinates:
  * lat      (lat) float64 53.63
  * lon      (lon) float64 -106.2
  * tile     (tile) int32 1
Attributes:
    units:      m
    long_name:  Soil permeable depth

### Check maximum level before bedrock
https://gitlab.com/jormelton/classic/-/blob/develop/src/modelStateDrivers.f90?ref_type=heads#L968

In [12]:
ds.DELZ

<xarray.DataArray 'DELZ' (layer: 20)>
array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.2,
        0.3,  0.4,  0.5,  1. ,  3. ,  5. , 15. , 30. ,  5. ])
Coordinates:
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Attributes:
    units:      m
    long_name:  Ground layer thickness

In [13]:
i = 0
while ds.DELZ.cumsum()[i] < SDEP:
    i += 1
    if i > ds.DELZ.shape[0]-1:
        print('The permable depth is greater than the model levels')
        break
        
maxlevel = i+1 # first level of bedrock (next to the one containing SDEP)
maxlevel

19

In [14]:
ds.DELZ.cumsum()

<xarray.DataArray 'DELZ' (layer: 20)>
array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.2,
        1.5,  1.9,  2.4,  3.4,  6.4, 11.4, 26.4, 56.4, 61.4])
Coordinates:
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20

In [15]:
ds.DELZ.cumsum()[maxlevel]

<xarray.DataArray 'DELZ' ()>
array(61.4)
Coordinates:
    layer    int32 20

In [16]:
ds.DELZ.cumsum()[:maxlevel]

<xarray.DataArray 'DELZ' (layer: 19)>
array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.2,
        1.5,  1.9,  2.4,  3.4,  6.4, 11.4, 26.4, 56.4])
Coordinates:
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19

In [17]:
ds.DELZ.cumsum()[maxlevel:]

<xarray.DataArray 'DELZ' (layer: 1)>
array([61.4])
Coordinates:
  * layer    (layer) int32 20

### Set soil contents and flags
Note: flags are only set in the SAND variable (-3 bedrock / -2 peatland)

In [18]:
with xr.set_options(keep_attrs=True):
    # Set values until bedrock
    ds.SAND[0, :maxlevel, 0, 0] = SAND[:maxlevel]
    ds.CLAY[0, :maxlevel, 0, 0] = CLAY[:maxlevel]
    ds.ORGM[0, :maxlevel, 0, 0] = ORGM[:maxlevel]
    
    # Set bedrock values
    ds.SAND[0, maxlevel:, 0, 0] = -3 # flag for bedrock
    ds.CLAY[0, maxlevel:, 0, 0] = 0.
    ds.ORGM[0, maxlevel:, 0, 0] = 0.

In [19]:
# Current values
ds.SAND[0, :, 0, 0]

<xarray.DataArray 'SAND' (layer: 20)>
array([-2. , 48.8, 48.8, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6,
       53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, 53.6, -3. ])
Coordinates:
    lat      float64 53.63
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
    lon      float64 -106.2
    tile     int32 1
Attributes:
    units:      %
    long_name:  Percentage sand content

In [20]:
ds.CLAY[0, :, 0, 0]

<xarray.DataArray 'CLAY' (layer: 20)>
array([ 0. , 24.2, 24.2, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1,
       22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1, 22.1,  0. ])
Coordinates:
    lat      float64 53.63
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
    lon      float64 -106.2
    tile     int32 1
Attributes:
    units:      %
    long_name:  Percentage clay content

In [21]:
ds.ORGM[0, :, 0, 0]

<xarray.DataArray 'ORGM' (layer: 20)>
array([77.5,  0.8,  0.8,  4.1,  4.1,  4.1,  4.1,  4.1,  4.1,  4.1,  4.1,
        4.1,  4.1,  4.1,  4.1,  4.1,  4.1,  4.1,  4.1,  0. ])
Coordinates:
    lat      float64 53.63
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
    lon      float64 -106.2
    tile     int32 1
Attributes:
    units:      %
    long_name:  Percentage organic matter content

In [22]:
ds

<xarray.Dataset>
Dimensions:            (tile: 1, lat: 1, lon: 1, icp1: 6, layer: 20, ic: 5,
                        icc: 12, iccp1: 13, iccp2: 14, slope: 8)
Coordinates:
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * icp1               (icp1) int32 1 2 3 4 5 6
  * lat                (lat) float64 53.63
  * layer              (layer) int32 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
  * lon                (lon) float64 -106.2
  * slope              (slope) int32 1 2 3 4 5 6 7 8
  * tile               (tile) int32 1
Data variables: (12/101)
    ALBS               (tile, lat, lon) float64 0.7647
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.29 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.05 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 0.0 24.2 ... 22.1 0.0
    CMAS               (tile, ic, lat, lon) float64 0.0 20.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    ...                 ...
    stemmass_s         (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    stmhrlos           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    tcoldm             (lat, lon) float32 99.0
    twarmm             (lat, lon) float32 -99.0
    tymaxlai           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    title:          CLASSIC initialization file created from: /home/acrp001/m...
    creation_date:  20190523
    history:        Wed Jun 12 17:19:01 2019: ncks -4 ./rsfile.nc ./rsfile.nc
    NCO:            4.4.2
    row_bounds:     1 1

## Save to netCDF

In [23]:
ds.to_netcdf(path_out+'/'+site+'_init_spinup_Paul.nc')
ds.to_netcdf(path_out+'/rsfile_spinup_Paul.nc')

In [24]:
!mkdir /home/lalandmi/eccc/classic-develop/outputFiles/SnowMIP/{site}/spinup_Paul